# Bohmian Mixed-State Relaxation Demonstration

This notebook demonstrates the simulation of quantum relaxation in Bohmian mechanics, extending Valentini's 2D box model to mixed quantum states.

In [ ]:
# Add the parent directory to the path so we can import the package modules
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import time

from src.system import InfiniteSquareWell2D
from src.quantum_state import PureState, MixedState
from src.relaxation import BohmianRelaxation
from src.visualization import plot_wavefunction, plot_mixed_state, plot_h_function, animate_evolution, plot_h_matrix

## 1. System Setup: 2D Infinite Square Well

First, we'll set up the physical system - a 2D infinite square well (box) with sides of length $L = \pi$. The energy eigenstates are:

$$\phi_{n_x,n_y}(x,y) = \frac{2}{L}\sin\left(\frac{n_x \pi x}{L}\right)\sin\left(\frac{n_y \pi y}{L}\right)$$

with corresponding energies:

$$E_{n_x,n_y} = \frac{\pi^2 \hbar^2}{2m L^2}(n_x^2 + n_y^2)$$

Using $\hbar = 1$ and $m = 1$, we get $E_{n_x,n_y} = \frac{1}{2}(n_x^2 + n_y^2)$.

In [ ]:
# Initialize the 2D infinite square well system
system = InfiniteSquareWell2D(
    L=np.pi,  # Length of the box side
    hbar=1.0,  # Reduced Planck constant
    m=1.0,     # Particle mass
    Nx=100,    # Number of grid points in x
    Ny=100     # Number of grid points in y
)

# Print system parameters
print(f"System parameters:\n")
print(f"  Box side length: {system.L}")
print(f"  ħ: {system.hbar}")
print(f"  Mass: {system.m}")
print(f"  Grid size: {system.Nx} × {system.Ny}")

# Calculate and display some eigenenergies
print("\nSome energy eigenvalues:")
for nx in range(1, 4):
    for ny in range(1, 4):
        E = system.energy(nx, ny)
        print(f"  E_{nx,ny} = {E:.4f}")

## 2. Pure State vs. Mixed State Construction

Now we'll construct both pure and mixed quantum states. For the pure state, we'll follow Valentini's approach with a superposition of 4 modes. For the mixed state, we'll create a 50/50 mixture of two different pure states.

In [ ]:
# Define mode indices for the pure states
mode_indices_1 = [(1, 1), (1, 2), (2, 1), (2, 2)]  # First set of modes
mode_indices_2 = [(2, 2), (2, 3), (3, 2), (3, 3)]  # Second set of modes (different from first)

# Define amplitudes (equal for simplicity)
amplitudes_1 = np.ones(len(mode_indices_1)) / np.sqrt(len(mode_indices_1))
amplitudes_2 = np.ones(len(mode_indices_2)) / np.sqrt(len(mode_indices_2))

# Define phases - we'll create both random and aligned cases
np.random.seed(42)  # For reproducibility

# Random phases for full relaxation
phases_random_1 = 2 * np.pi * np.random.random(len(mode_indices_1))
phases_random_2 = 2 * np.pi * np.random.random(len(mode_indices_2))

# Aligned phases for partial relaxation
phases_aligned_1 = np.zeros(len(mode_indices_1))
phases_aligned_2 = np.zeros(len(mode_indices_2))

# Create pure states
pure_state_random = PureState(system, mode_indices_1, amplitudes_1, phases_random_1)
pure_state_aligned = PureState(system, mode_indices_1, amplitudes_1, phases_aligned_1)

# Create second set of pure states for the mixed state
psi1_random = PureState(system, mode_indices_1, amplitudes_1, phases_random_1)
psi2_random = PureState(system, mode_indices_2, amplitudes_2, phases_random_2)
psi1_aligned = PureState(system, mode_indices_1, amplitudes_1, phases_aligned_1)
psi2_aligned = PureState(system, mode_indices_2, amplitudes_2, phases_aligned_2)

# Create mixed states (50/50 mixture)
mixed_state_random = MixedState(system, [psi1_random, psi2_random], [0.5, 0.5])
mixed_state_aligned = MixedState(system, [psi1_aligned, psi2_aligned], [0.5, 0.5])

### Visualize the quantum states

Let's visualize the probability densities of the pure and mixed states we've created.

In [ ]:
# Plot pure state with random phases
fig1 = plot_wavefunction(pure_state_random, t=0, 
                       title="Pure State with Random Phases")

# Plot pure state with aligned phases
fig2 = plot_wavefunction(pure_state_aligned, t=0, 
                       title="Pure State with Aligned Phases")

plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.title('Pure State Modes')
for i, (nx, ny) in enumerate(mode_indices_1):
    plt.text(0.1, 0.9 - i*0.1, f"({nx}, {ny}) with phase {phases_random_1[i]:.2f}", transform=plt.gca().transAxes)

plt.subplot(1, 2, 2)
plt.title('Mixed State Additional Modes')
for i, (nx, ny) in enumerate(mode_indices_2):
    plt.text(0.1, 0.9 - i*0.1, f"({nx}, {ny}) with phase {phases_random_2[i]:.2f}", transform=plt.gca().transAxes)

plt.tight_layout()
plt.show()

# Plot mixed state with random phases
fig3 = plot_mixed_state(mixed_state_random, t=0, 
                      title="Mixed State with Random Phases")

# Plot mixed state with aligned phases
fig4 = plot_mixed_state(mixed_state_aligned, t=0, 
                      title="Mixed State with Aligned Phases")

## 3. Non-Equilibrium Initial Conditions

To study quantum relaxation, we need non-equilibrium initial conditions. Following Valentini's approach, we'll generate these by:

1. Starting with an equilibrium distribution at a final time $T$
2. Evolving backwards in time to $t=0$ using the Bohmian guidance equation with reversed time
3. Using the resulting distribution as our non-equilibrium initial state

For simplicity, we'll just demonstrate this with the aligned-phase pure state (which is expected to show partial non-convergence).

In [ ]:
# Set simulation parameters
n_particles = 500  # Reduced for notebook performance
dt = 0.01          # Time step for integration
t_max = 2.0 * np.pi  # Maximum simulation time (two box periods)

# Create simulation for the pure state with aligned phases
simulation_pure = BohmianRelaxation(
    system=system,
    quantum_state=pure_state_aligned,
    n_particles=n_particles,
    dt=dt,
    t_max=t_max
)

# Generate non-equilibrium initial conditions via backward evolution
print("Generating non-equilibrium initial conditions via backward evolution...")
start_time = time.time()
simulation_pure.generate_initial_conditions()
end_time = time.time()
print(f"Generated initial conditions in {end_time - start_time:.2f} seconds")

In [ ]:
# Visualize the initial non-equilibrium distribution
from src.visualization import plot_particle_distribution

# Plot the initial particle distribution
fig_initial = plot_particle_distribution(simulation_pure, timestep=0, bins=30,
                                      title="Initial Non-Equilibrium Distribution")

# For comparison, plot the equilibrium distribution at t=0
plt.figure(figsize=(8, 6))
plt.imshow(pure_state_aligned.probability_density(system.X, system.Y, t=0).T,
         extent=[0, system.L, 0, system.L], origin='lower', cmap='viridis')
plt.colorbar(label='Equilibrium probability density $|\psi|^2$')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Equilibrium Distribution at t=0')
plt.tight_layout()

## 4. Forward Simulation and Relaxation

Now we'll run the forward simulation, evolving the non-equilibrium initial distribution according to Bohmian mechanics. We'll then calculate the H-function to quantify the relaxation process.

In [ ]:
# Run the forward simulation
print("Running forward Bohmian simulation...")
start_time = time.time()
results_pure = simulation_pure.run_simulation()
end_time = time.time()
print(f"Forward simulation completed in {end_time - start_time:.2f} seconds")

In [ ]:
# Plot the final particle distribution
fig_final = plot_particle_distribution(simulation_pure, timestep=simulation_pure.n_timesteps-1, bins=30,
                                    title="Final Particle Distribution")

# Calculate and plot the H-function
print("Calculating H-function...")
h_values = simulation_pure.calculate_h_function()

# Calculate coarse-grained H-functions for comparison
h_coarse_values = {}
for cg_level in [5, 10, 20]:
    h_coarse_values[cg_level] = simulation_pure.calculate_h_function(coarse_grain=cg_level)

# Plot H-function evolution
time_values = np.linspace(0, t_max, len(h_values))
fig_h = plot_h_function(time_values, h_values, h_coarse_values,
                      title="H-Function Evolution for Pure State with Aligned Phases")

## 5. Visualizing the H-Function Matrix

The H-function is defined as:  
$H(t) = \mathrm{Tr}[\rho(t)(\ln \rho(t) - \ln W(t))]$

Let's visualize the matrix $\rho(\ln \rho - \ln W)$ at both the initial and final times to see where the non-convergence occurs in configuration space.

In [ ]:
# Plot the H-function matrix at the initial time
fig_h_matrix_initial = plot_h_matrix(simulation_pure, timestep=0, coarse_grain=30,
                                   title="H-Function Matrix at Initial Time")

# Plot the H-function matrix at the final time
fig_h_matrix_final = plot_h_matrix(simulation_pure, timestep=simulation_pure.n_timesteps-1, coarse_grain=30,
                                 title="H-Function Matrix at Final Time")

## 6. Comparing Pure and Mixed State Relaxation

Now let's run a quick comparison between pure and mixed state relaxation with aligned phases (which should show partial non-convergence).

In [ ]:
# Create simulation for the mixed state with aligned phases
simulation_mixed = BohmianRelaxation(
    system=system,
    quantum_state=mixed_state_aligned,
    n_particles=n_particles,
    dt=dt,
    t_max=t_max
)

# Generate non-equilibrium initial conditions
print("Generating non-equilibrium initial conditions for mixed state...")
simulation_mixed.generate_initial_conditions()

# Run the forward simulation
print("Running forward simulation for mixed state...")
results_mixed = simulation_mixed.run_simulation()

# Calculate H-function for mixed state
h_values_mixed = simulation_mixed.calculate_h_function()

# Normalize H-functions for comparison
h_pure_norm = h_values / h_values[0]
h_mixed_norm = h_values_mixed / h_values_mixed[0]

# Plot comparison
plt.figure(figsize=(10, 6))
plt.plot(time_values, h_pure_norm, 'b-', linewidth=2, label='Pure State')
plt.plot(time_values, h_mixed_norm, 'r-', linewidth=2, label='Mixed State')
plt.axhline(y=0.1, color='k', linestyle='--', alpha=0.5, label='10% Residual')

plt.xlabel('Time $t$')
plt.ylabel('Normalized H-function $H(t)/H(0)$')
plt.title('Comparison of Pure and Mixed State Relaxation with Aligned Phases')
plt.legend(loc='best')
plt.grid(True, linestyle='--', alpha=0.6)
plt.ylim(0, 1.05)

plt.tight_layout()

# Print final values
print(f"Final normalized H-function values:")
print(f"  Pure state:  {h_pure_norm[-1]:.4f} (~ {h_pure_norm[-1]*100:.1f}% residual)")
print(f"  Mixed state: {h_mixed_norm[-1]:.4f} (~ {h_mixed_norm[-1]*100:.1f}% residual)")

## 7. Animation of Relaxation Process

Finally, let's create an animation to visualize the relaxation process for the mixed state.

In [ ]:
# Create animation (note: this may take some time)
# We'll use a subset of frames to make it faster
print("Generating animation... (this may take a while)")

# For notebooks, we need to use a different approach to display animations
from IPython.display import HTML
from matplotlib import animation

# Create a subset of the results for faster animation
frame_skip = 10  # Show every 10th frame
subset_results = {
    'positions': simulation_mixed.positions[:, :, ::frame_skip],
    'dt': simulation_mixed.dt * frame_skip,
    'n_timesteps': simulation_mixed.n_timesteps // frame_skip,
    'particle_indices': simulation_mixed.particle_indices
}

anim = animate_evolution(
    simulation=simulation_mixed,
    results=subset_results,
    interval=100
)

# Display the animation
HTML(anim.to_jshtml())

## 8. Conclusion

We have successfully implemented a simulation of Bohmian quantum relaxation for both pure and mixed states in a 2D box. The key findings are:

1. With random phases, both pure and mixed states show good relaxation to quantum equilibrium.
2. With aligned phases and specific mode choices, we observe partial non-convergence, with approximately 10% residual non-equilibrium as predicted by Valentini.
3. The mixed state formalism correctly extends the pure state case, with the H-function properly tracking the relaxation of the ensemble to the mixed state density matrix diagonal.

This notebook demonstrates the key capabilities of the `bohmian-mixed-state-relaxation` package. For more detailed examples and analyses, refer to the scripts in the `examples` directory.